# Preprocessing, GridSearch and Training DenseNet-8 on Open data

In [1]:
import numpy as np
from numpy import array
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import math

import matplotlib.pylab as plt
from matplotlib.pyplot import cm
import seaborn as sns
import itertools
import collections
from collections import OrderedDict, Counter

import scipy.stats as stats
from scipy.stats import normaltest, kstest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, levene, bartlett
from scipy.stats import chi2_contingency, fisher_exact, mode, pearsonr, f_oneway, kruskal, spearmanr

from rfpimp import *

from datetime import datetime
import re
from seaborn import heatmap
import random
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.utils import resample
from functools import reduce
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, recall_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PowerTransformer

from sklearn.utils import check_consistent_length, column_or_1d, assert_all_finite
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression

import gc
import os

#from sklearn.externals import joblib
import joblib

from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import stratified_split
from Filter_and_Grid_Search import attributes_list, attributes_list_new
from Filter_and_Grid_Search import get_s_stat, get_PSI_stat, get_stats_by_month, get_stats, stable_unstable
from Filter_and_Grid_Search import stable_unstable_by_month_divide, union_datas, individual_hists_all 
from Filter_and_Grid_Search import paired_time_hists_by_month, statistics_with_target
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import statistics_with_target, attributes_list, attributes_list_new, make_standard
from Filter_and_Grid_Search import data_preprocessing_train, data_preprocessing_test
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import stratified_split, two_forests, turn_variables_with_values
from Filter_and_Grid_Search import find_meta_params, calculate_vif#, find_meta_params_mem
from Filter_and_Grid_Search import plot_meta_2d, data_preprocessing, find_ouliers_iqr
from Filter_and_Grid_Search import train_model_receive_stats, simple_b_score_risk
from Filter_and_Grid_Search import max_prof_corve, by_month_gini, check_attribute_list_cases

from Filter_and_Grid_Search import to_zip, br_correction, br_stat

from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pathlib
import psutil

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


# ***

## Open data

## Loading preprocessed data

In [3]:
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_chinese/'

In [4]:
target = 'LABEL'
index_month = 'MONTH'
list_of_vars_for_strat = ['MONTH']
sort_by_var = 'ID'

necessary_fields = [target, index_month, sort_by_var]

COL_DEL = ['Unnamed: 0'] 
COL_DEL = [x.upper() for x in COL_DEL]
COL_TRG = target


In [5]:
train_for = pd.read_csv(PATH + 'china_train_128_prep.csv')
valid_for = pd.read_csv(PATH + 'china_val_128_prep.csv')
test_for = pd.read_csv(PATH + 'china_test_128_prep.csv')


y_train = train_for[target]
y_test = valid_for[target]
y_val = test_for[target]

train_for.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for.drop('Unnamed: 0', axis = 1, inplace = True)
test_for.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for.columns.to_list()
col.remove(target)


X_1_2 = train_for[col]
X_2_2 = valid_for[col]
X_3_2 = test_for[col]



In [6]:
x_col = X_2_2.columns
y_col = 'LABEL'

In [7]:
#проверка корректности
X_1_2.shape, train_for.shape, len(y_train)

((792004, 128), (792004, 129), 792004)

In [8]:
#проверка корректности, должно быть везде True
print(X_1_2.shape[0] == train_for.shape[0])
print(X_1_2.shape[1] == train_for.shape[1] - 1)
print(len(y_train) == train_for.shape[0])

True
True
True


# ***

## Tensorflow

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Dropout)
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, AveragePooling1D)
from tensorflow.keras.layers import add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

In [11]:
from tensorflow.keras.layers import ReLU, Concatenate, GlobalAveragePooling1D

In [12]:
from keras.utils import np_utils
from tensorflow.keras.models import Model

In [13]:
# Задаем seed_value для воспроизводимости
# Можно использовать разные seed_value для разных систем
seed_value= 29


# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)


# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, Callback, LearningRateScheduler

# объявляем класс метрик

class E_time(Callback):
    def on_train_begin(self, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('train_begin', '| time: ' , tm)
        
    def on_epoch_end(self, epoch, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('epoch_end', '| time: ' , tm)
    
        return

_time = E_time()

In [15]:
#если alpha не задано, то не делаем br_correction

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        return [np.array(X)], np.array(y)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

In [16]:
# здесь батчсайз надо указывать тот, который получился по гридсерч сетке
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05)

In [17]:
Y_test_2 = np_utils.to_categorical(y_test, 2) # преобразовываем в 2 класса 

In [18]:
sum(y_train)/y_train.shape[0]

0.012246150271968324

# DenseNet-8 (1D)

In [19]:
import tensorflow
from tensorflow.keras.layers import BatchNormalization, Activation, Conv1D, Concatenate, AveragePooling1D

In [20]:
# функции для 1D-DenseNet
# надо по гридсерч сетке подобрать параметры:
# 1) k (количетсво фильтров): сделать параметры адекватные для 1D (по умолчанию стоят от 2D)
# 2) kernel_width (размер фильтра/свертки): сделать параметры адекватные для 1D (по умолчанию стоят от 2D)
# 3) bottleneck_size - это мультпликатор для кол-ва фильтров k.
# 4) num_layers - количество сверток в денз-блоке (нужно 3 и 4 - соотвествует спайдеру/2)

""" 
blocks/one_d.py
Author: Ankit Gupta

Implementations of various DenseNet blocks for 1D sequences

This module contains helper functions that define the various subcomponents of a DenseNet. 
This includes dense blocks and transition blocks.

"""

#from tf.keras.layers import BatchNormalization, Activation, Conv1D, Concatenate, AveragePooling1D


def H_l(k, bottleneck_size, kernel_width):
    """ 
    A single convolutional "layer" as defined by Huang et al. Defined as H_l in the original paper
    
    :param k: int representing the "growth rate" of the DenseNet
    :param bottleneck_size: int representing the size of the bottleneck, as a multiple of k. 
    Set to 0 for no bottleneck.
    :param kernel_width: int representing the width of the main convolutional kernel
    :return a function wrapping the keras layers for H_l
    """

    use_bottleneck = bottleneck_size > 0
    num_bottleneck_output_filters = k * bottleneck_size

    def f(x):
        if use_bottleneck:
            x = BatchNormalization()(x)
            x = Activation("relu")(x)
            x = Conv1D(
                num_bottleneck_output_filters,
                1,
                strides=1,
                padding="same",
                dilation_rate=1)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv1D(
            k,
            kernel_width,
            strides=1,
            padding="same",
            dilation_rate=1)(x)
        return x
    return f


def dense_block(k, num_layers, kernel_width, bottleneck_size):
    """
    A single dense block of the DenseNet
    
    :param k: int representing the "growth rate" of the DenseNet
    :param num_layers: int represending the number of layers in the block
    :param kernel_width: int representing the width of the main convolutional kernel
    :param bottleneck_size: int representing the size of the bottleneck, as a multiple of k. Set to 0 for no bottleneck.
    :return a function wrapping the entire dense block
    """
    def f(x):
        layers_to_concat = [x]
        for _ in range(num_layers):
            x = H_l(k, bottleneck_size, kernel_width)(x)
            layers_to_concat.append(x)
            x = Concatenate(axis=-1)(layers_to_concat)
        return x
    return f


def transition_block(pool_size=2, stride=2, theta=0.5):
    """
    A single transition block of the DenseNet
    
    :param pool_size: int represending the width of the average pool
    :param stride: int represending the stride of the average pool
    :param theta: int representing the amount of compression in the 1x1 convolution. Set to 1 for no compression.
    :return a function wrapping the entire transition block
    """    
    assert theta > 0 and theta <= 1

    def f(x):
        num_transition_output_filters = int(int(x.shape[2]) * float(theta))
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv1D(
            num_transition_output_filters,
            1,
            strides=1,
            padding="same",
            dilation_rate=1)(x)
        x = AveragePooling1D(
            pool_size=pool_size,
            strides=stride,
            padding="same")(x)
        return x
    return f


def DenseNet_1(
        k,
        block_sizes,
        conv_kernel_width,
        bottleneck_size,
        transition_pool_size,
        transition_pool_stride,
        theta,
        initial_conv_width,
        initial_stride,
        initial_filters,
        initial_pool_width,
        initial_pool_stride,
        use_global_pooling):
    def f(x):
        x = Conv1D(
            initial_filters,
            initial_conv_width,
            strides=initial_stride,
            padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = MaxPooling1D(
            pool_size=initial_pool_width,
            strides=initial_pool_stride,
            padding="same")(x)

        # Add all but the last dense block
        for block_size in block_sizes[:-1]:
            x = dense_block(
                k,
                block_size,
                conv_kernel_width,
                bottleneck_size)(x)
            x = transition_block(
                pool_size=transition_pool_size,
                stride=transition_pool_stride,
                theta=theta)(x)

        # Add the last dense block
        final_block_size = block_sizes[-1]
        x = dense_block(
            k,
            final_block_size,
            conv_kernel_width,
            bottleneck_size)(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        if use_global_pooling:
            x = GlobalAveragePooling1D()(x)
        return x
    return f



class DenseNetCustom(tensorflow.keras.models.Model):
    """  
    Create a Keras Model Object that is an implementation of DenseNet with a custom number of parameters. The number of layers 
    per dense block can be specified by block_sizes.
    :param input_shape: The shape of the inputs without the batch dimension. This should be a valid 1D sequence, 
    such as (244, 25). 
    :param num_outputs: the number of classes to predict
    :param k: The "growth rate" of the DenseNet model
    :param block_sizes: A list of ints with the number of layers in each block. Example: [5, 10, 25, 17].
    :param conv_kernel_width: The kernel width of each convolution in the dense blocks.
    :param bottleneck_size: The size of the bottleneck, as a multiple of k. Set to 0 for no bottleneck.
    :param transition_pool_size: pool_size in the transition layer
    :param transition_pool_stride: pooling stride in the transition layer
    :param theta: Amount of compression in the transition layer. Set to 1 for no compression.
    :param initial_conv_width: Kernel width for the one convolution before the dense blocks
    :param initial_stride: Stride for the one convolution before the dense blocks
    :param initial_filters: Number of filters for the one convolution before the dense blocks
    :param initial_pool_width: pool_size for the one pooling before the dense blocks
    :param initial_pool_stride: stride for the one pooling before the dense blocks 
    """
    def __init__(
            self,
            input_shape,
            num_outputs=1000,  # здесь нужно 2 (1000 классов - это из ImageNet)
            k=32,  # количество фильтров нужно 5-10
            block_sizes=None,
            conv_kernel_width=3,  # размер фильтра нужно 5 (можно подобрать). 3х3 - это из 2D взяли.
            bottleneck_size=4,  # этот мультипликатор для k лучше взять 2-3(общая формула Х = initial_filters + k * bottleneck_size (k={0,n}))
            transition_pool_size=2,
            transition_pool_stride=2, # здесь лучше поставить страйд = 1 (2 - из 2D)
            theta=0.5, # это параметр для баттлнек сверток 1х1, мб настроить парочку из списка: {0.4, 0.5, 0.6, 1}
            initial_conv_width=7,  # здесь лучше поставить 5 (7 - для 2D)
            initial_stride=2, # здесь лучше 1 взять (2 - из 2D)
            initial_filters=64,  # количество фильтров лучше взять 10-20 (64 - из 2D)
            initial_pool_width=3, # этот пулинг лучше 2 взять (3 - из 2D)
            initial_pool_stride=2):  # здесь лучше поставить страйд = 1, а то размерность будет быстро уменьшаться
        if not block_sizes:
            raise ValueError("block_sizes must be specified")
        model_input = Input(shape=input_shape)
        output = DenseNet_1(
            k,
            block_sizes,
            conv_kernel_width,
            bottleneck_size,
            transition_pool_size,
            transition_pool_stride,
            theta,
            initial_conv_width,
            initial_stride,
            initial_filters,
            initial_pool_width,
            initial_pool_stride,
            use_global_pooling=True)(model_input)
        output = Dense(num_outputs, activation="softmax")(output)
        super(DenseNetCustom, self).__init__(model_input, output)


In [21]:
# Гридсерч сетку настроить:
# k = {5, 10}
# initial_filters = {5, 10}, # количество фильтров лучше взять 5-10-20 (64 - из 2D)
# block_sizes = [3, 3] - для Spider-6; [4, 4] - для Spider-8
# conv_kernel_width = w, где for w in [5, 6] 
# bottleneck_size=2, лучше взять 2-3(общая формула Х = initial_filters + k * bottleneck_size (k={0,n}))
# transition_pool_size=2,
# transition_pool_stride=2, # здесь лучше поставить страйд = 1 (2 - из 2D)
# theta=0.5, # это параметр для баттлнек сверток 1х1, мб настроить парочку из списка: {0.4, 0.5, 0.6, 1}
# initial_conv_width=5,  # здесь лучше поставить 5 (7 - для 2D)
# initial_stride=2, # здесь лучше 1 взять (2 - из 2D)
# initial_pool_width=3, # этот пулинг лучше 2 взять (3 - из 2D)
# initial_pool_stride=2  # здесь лучше поставить страйд = 1, а то размерность будет быстро уменьшаться


n_features = 128
model_dense = DenseNetCustom(input_shape = (n_features, 1),
                             num_outputs=2,
                             block_sizes= [3, 3],
                             initial_filters=5,
                             k=10, 
                             conv_kernel_width=5,
                             bottleneck_size=2,
                             transition_pool_stride=1,
                             theta=0.5,
                             initial_conv_width=5,
                             initial_stride=1,
                             initial_pool_width=2,
                             initial_pool_stride=2)

In [22]:
model_dense.compile(loss='categorical_crossentropy', #optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              #optimizer='adam', 
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model_dense.summary()

Model: "dense_net_custom"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 1)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 128, 5)       30          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 5)       20          conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 5)       0           batch_normalization[0][0]        
___________________________________________________________________________________

In [23]:
import os
os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin\\'

In [25]:
# здесь батчсайз и class_w надо указывать тот, который получился по гридсерч сетке
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.1)

In [26]:
# anealing - снижение ленингрэйта (на Спайдере не дает доп.эффекта)
#learning_rate=0.005
initial_learning_rate = 0.05

def lr_step_decay(epoch, lr):
    drop_rate = 0.2
    epochs_drop = 20.0
    
    return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))

## GridSearch for DenseNet-8

In [ ]:
# Гридсерч сетку настроить:
# k = {5, 10}
# initial_filters = {5, 10}, # количество фильтров лучше взять 5-10-20 (64 - из 2D)
# block_sizes = [3, 3] - для Spider-6; [4, 4] - для Spider-8
# conv_kernel_width = w, где for w in [5, 6] 
# bottleneck_size=2, лучше взять 2-3(общая формула Х = initial_filters + k * bottleneck_size (k={0,n}))
# transition_pool_size=2,
# transition_pool_stride=2, # здесь наверно лучше поставить страйд = 1 (2 - из 2D наверно)
# theta=0.5, # это непонятный параметр для баттлнек сверток 1х1, мб настроить парочку из списка: {0.4, 0.5, 0.6, 1}
# initial_conv_width=5,  # здесь лучше поставить 5 (7 - для 2D)
# initial_stride=2, # здесь лучше 1 взять (2 - наверно из 2D)
# initial_pool_width=3, # этот пулинг лучше 2 взять (3 наверно из 2D)
# initial_pool_stride=2  # здесь наверно лучше поставить страйд = 1, а то размерность будет быстро уменьшаться


n_features = 163

for k_i in [5, 10]:
    for theta_i in [0.5, 1]:
        for w in [3, 5]:
            model_dense = DenseNetCustom(input_shape = (163, 1),
                                                 num_outputs=2,
                                                 block_sizes= [3, 3],
                                                 initial_filters=5,
                                                 k=k_i, 
                                                 conv_kernel_width=5,
                                                 bottleneck_size=2,
                                                 transition_pool_stride=1,
                                                 theta=theta_i,
                                                 initial_conv_width=w,
                                                 initial_stride=1,
                                                 initial_pool_width=2,
                                                 initial_pool_stride=2)

            model_dense.compile(loss='categorical_crossentropy',
                                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                                        metrics=['accuracy', tf.keras.metrics.AUC(),
                                                 tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

            VERBOSE = 2 # формат вывода логов обучения
            BATCH_SIZE = 512
            NB_EPOCH = 100 # максимальное количество эпох, если не сработает EarlyStopping
            class_weighting = {0:0.1 , 1:1} #{0:0.00163 , 1:1} можно подбирать как гипер.параметр.

            history_logs = model_dense.fit_generator(generator=training_generator,
                                                             validation_data= (X_2_2, Y_test_2),
                                                             epochs=NB_EPOCH, verbose=VERBOSE,
                                                             class_weight=class_weighting,
                                                             callbacks=[_time, EarlyStopping(monitor='val_loss', patience=15)])

            res_model_dense_3_3 = pd.DataFrame(history_logs.history, columns = history_logs.history.keys())
            model_dense.save('model_dense_OpenData_c3c3' + '_k' + str(k_i) + '_t' + str(theta_i) + '_w' + str(w) + '.h5')
            res_model_dense_3_3.to_csv('model_dense_OpenData_c3c3' + '_k' + str(k_i) + '_t' + str(theta_i) + '_w' + str(w) + '.csv')

            predict_class_val = model_dense.predict(X_2_2)
            APS = metrics.average_precision_score(y_test, predict_class_val[:,1])
            AUC = metrics.roc_auc_score(y_test, predict_class_val[:,1])
            GINI = 2*AUC - 1
            print('=========================================')
            print('VALIDATION:', 'APS:', APS, 'AUC_ROC:', AUC, 'GINI:', GINI)

            predict_class_val = model_dense.predict(X_3_2)
            APS = metrics.average_precision_score(y_val, predict_class_val[:,1])
            AUC = metrics.roc_auc_score(y_val, predict_class_val[:,1])
            GINI = 2*AUC - 1
            print('TEST:', 'APS:', APS, 'AUC_ROC:', AUC, 'GINI:', GINI)
            print('=========================================')
            print('')
                    



train_begin | time:  12.04.2021 15:50:27
Epoch 1/100
epoch_end | time:  12.04.2021 16:06:28
1547/1547 - 956s - loss: 0.0330 - accuracy: 0.9285 - auc_1: 0.9773 - precision_1: 0.9285 - recall_1: 0.9285 - val_loss: 0.2182 - val_accuracy: 0.9270 - val_auc_1: 0.9756 - val_precision_1: 0.9270 - val_recall_1: 0.9270
Epoch 2/100
epoch_end | time:  12.04.2021 16:21:18
1547/1547 - 890s - loss: 0.0231 - accuracy: 0.9568 - auc_1: 0.9878 - precision_1: 0.9568 - recall_1: 0.9568 - val_loss: 0.1318 - val_accuracy: 0.9563 - val_auc_1: 0.9881 - val_precision_1: 0.9563 - val_recall_1: 0.9563
Epoch 3/100
epoch_end | time:  12.04.2021 16:34:28
1547/1547 - 789s - loss: 0.0211 - accuracy: 0.9618 - auc_1: 0.9896 - precision_1: 0.9618 - recall_1: 0.9618 - val_loss: 0.0736 - val_accuracy: 0.9766 - val_auc_1: 0.9955 - val_precision_1: 0.9766 - val_recall_1: 0.9766
Epoch 4/100
epoch_end | time:  12.04.2021 16:49:41
1547/1547 - 912s - loss: 0.0200 - accuracy: 0.9641 - auc_1: 0.9906 - precision_1: 0.9641 - recall_

Epoch 31/100
epoch_end | time:  12.04.2021 22:41:05
1547/1547 - 717s - loss: 0.0146 - accuracy: 0.9761 - auc_1: 0.9948 - precision_1: 0.9761 - recall_1: 0.9761 - val_loss: 0.1208 - val_accuracy: 0.9613 - val_auc_1: 0.9893 - val_precision_1: 0.9613 - val_recall_1: 0.9613
Epoch 32/100
epoch_end | time:  12.04.2021 22:53:06
1547/1547 - 720s - loss: 0.0145 - accuracy: 0.9764 - auc_1: 0.9949 - precision_1: 0.9764 - recall_1: 0.9764 - val_loss: 0.0666 - val_accuracy: 0.9793 - val_auc_1: 0.9961 - val_precision_1: 0.9793 - val_recall_1: 0.9793
Epoch 33/100
epoch_end | time:  12.04.2021 23:05:04
1547/1547 - 718s - loss: 0.0145 - accuracy: 0.9764 - auc_1: 0.9949 - precision_1: 0.9764 - recall_1: 0.9764 - val_loss: 0.0822 - val_accuracy: 0.9733 - val_auc_1: 0.9944 - val_precision_1: 0.9733 - val_recall_1: 0.9733
Epoch 34/100
epoch_end | time:  12.04.2021 23:17:03
1547/1547 - 718s - loss: 0.0144 - accuracy: 0.9766 - auc_1: 0.9949 - precision_1: 0.9766 - recall_1: 0.9766 - val_loss: 0.0703 - val_ac

Epoch 17/100
epoch_end | time:  13.04.2021 04:13:05
1547/1547 - 675s - loss: 0.0162 - accuracy: 0.9730 - auc_2: 0.9935 - precision_2: 0.9730 - recall_2: 0.9730 - val_loss: 0.0895 - val_accuracy: 0.9712 - val_auc_2: 0.9936 - val_precision_2: 0.9712 - val_recall_2: 0.9712
Epoch 18/100
epoch_end | time:  13.04.2021 04:24:22
1547/1547 - 676s - loss: 0.0161 - accuracy: 0.9729 - auc_2: 0.9935 - precision_2: 0.9729 - recall_2: 0.9729 - val_loss: 0.1480 - val_accuracy: 0.9527 - val_auc_2: 0.9843 - val_precision_2: 0.9527 - val_recall_2: 0.9527
Epoch 19/100
epoch_end | time:  13.04.2021 04:35:35
1547/1547 - 673s - loss: 0.0161 - accuracy: 0.9732 - auc_2: 0.9936 - precision_2: 0.9732 - recall_2: 0.9732 - val_loss: 0.0815 - val_accuracy: 0.9748 - val_auc_2: 0.9943 - val_precision_2: 0.9748 - val_recall_2: 0.9748
Epoch 20/100
epoch_end | time:  13.04.2021 04:46:52
1547/1547 - 676s - loss: 0.0159 - accuracy: 0.9734 - auc_2: 0.9937 - precision_2: 0.9734 - recall_2: 0.9734 - val_loss: 0.0968 - val_ac

Epoch 20/100
epoch_end | time:  13.04.2021 10:15:01
1547/1547 - 850s - loss: 0.0145 - accuracy: 0.9763 - auc_3: 0.9948 - precision_3: 0.9763 - recall_3: 0.9763 - val_loss: 0.0809 - val_accuracy: 0.9765 - val_auc_3: 0.9939 - val_precision_3: 0.9765 - val_recall_3: 0.9765
Epoch 21/100
epoch_end | time:  13.04.2021 10:29:18
1547/1547 - 857s - loss: 0.0145 - accuracy: 0.9761 - auc_3: 0.9949 - precision_3: 0.9761 - recall_3: 0.9761 - val_loss: 0.1234 - val_accuracy: 0.9632 - val_auc_3: 0.9883 - val_precision_3: 0.9632 - val_recall_3: 0.9632
Epoch 22/100
epoch_end | time:  13.04.2021 10:43:42
1547/1547 - 863s - loss: 0.0143 - accuracy: 0.9767 - auc_3: 0.9949 - precision_3: 0.9767 - recall_3: 0.9767 - val_loss: 0.1487 - val_accuracy: 0.9520 - val_auc_3: 0.9847 - val_precision_3: 0.9520 - val_recall_3: 0.9520
Epoch 23/100
epoch_end | time:  13.04.2021 10:57:48
1547/1547 - 845s - loss: 0.0143 - accuracy: 0.9769 - auc_3: 0.9950 - precision_3: 0.9769 - recall_3: 0.9769 - val_loss: 0.0856 - val_ac

Epoch 24/100
epoch_end | time:  13.04.2021 17:46:29
1547/1547 - 888s - loss: 0.0146 - accuracy: 0.9765 - auc_4: 0.9949 - precision_4: 0.9765 - recall_4: 0.9765 - val_loss: 0.0915 - val_accuracy: 0.9711 - val_auc_4: 0.9935 - val_precision_4: 0.9711 - val_recall_4: 0.9711
Epoch 25/100
epoch_end | time:  13.04.2021 18:02:42
1547/1547 - 973s - loss: 0.0145 - accuracy: 0.9767 - auc_4: 0.9949 - precision_4: 0.9767 - recall_4: 0.9767 - val_loss: 0.0660 - val_accuracy: 0.9808 - val_auc_4: 0.9960 - val_precision_4: 0.9808 - val_recall_4: 0.9808
Epoch 26/100
epoch_end | time:  13.04.2021 18:17:11
1547/1547 - 868s - loss: 0.0146 - accuracy: 0.9764 - auc_4: 0.9949 - precision_4: 0.9764 - recall_4: 0.9764 - val_loss: 0.0574 - val_accuracy: 0.9821 - val_auc_4: 0.9968 - val_precision_4: 0.9821 - val_recall_4: 0.9821
Epoch 27/100
epoch_end | time:  13.04.2021 18:32:19
1547/1547 - 907s - loss: 0.0144 - accuracy: 0.9768 - auc_4: 0.9951 - precision_4: 0.9768 - recall_4: 0.9768 - val_loss: 0.0648 - val_ac

Epoch 55/100
epoch_end | time:  14.04.2021 01:17:32
1547/1547 - 818s - loss: 0.0133 - accuracy: 0.9786 - auc_4: 0.9958 - precision_4: 0.9786 - recall_4: 0.9786 - val_loss: 0.0982 - val_accuracy: 0.9692 - val_auc_4: 0.9924 - val_precision_4: 0.9692 - val_recall_4: 0.9692
Epoch 56/100
epoch_end | time:  14.04.2021 01:31:11
1547/1547 - 819s - loss: 0.0132 - accuracy: 0.9788 - auc_4: 0.9958 - precision_4: 0.9788 - recall_4: 0.9788 - val_loss: 0.0644 - val_accuracy: 0.9802 - val_auc_4: 0.9961 - val_precision_4: 0.9802 - val_recall_4: 0.9802
Epoch 57/100
epoch_end | time:  14.04.2021 01:44:50
1547/1547 - 818s - loss: 0.0132 - accuracy: 0.9789 - auc_4: 0.9958 - precision_4: 0.9789 - recall_4: 0.9789 - val_loss: 0.0896 - val_accuracy: 0.9720 - val_auc_4: 0.9936 - val_precision_4: 0.9720 - val_recall_4: 0.9720
Epoch 58/100
epoch_end | time:  14.04.2021 01:58:26
1547/1547 - 815s - loss: 0.0132 - accuracy: 0.9787 - auc_4: 0.9958 - precision_4: 0.9787 - recall_4: 0.9787 - val_loss: 0.1020 - val_ac

Epoch 16/100
epoch_end | time:  14.04.2021 09:26:05
1547/1547 - 1208s - loss: 0.0122 - accuracy: 0.9800 - auc_5: 0.9962 - precision_5: 0.9800 - recall_5: 0.9800 - val_loss: 0.0882 - val_accuracy: 0.9733 - val_auc_5: 0.9934 - val_precision_5: 0.9733 - val_recall_5: 0.9733
Epoch 17/100
epoch_end | time:  14.04.2021 09:46:11
1547/1547 - 1205s - loss: 0.0120 - accuracy: 0.9803 - auc_5: 0.9964 - precision_5: 0.9803 - recall_5: 0.9803 - val_loss: 0.0595 - val_accuracy: 0.9819 - val_auc_5: 0.9964 - val_precision_5: 0.9819 - val_recall_5: 0.9819
Epoch 18/100
epoch_end | time:  14.04.2021 10:06:36
1547/1547 - 1223s - loss: 0.0119 - accuracy: 0.9803 - auc_5: 0.9965 - precision_5: 0.9803 - recall_5: 0.9803 - val_loss: 0.0610 - val_accuracy: 0.9818 - val_auc_5: 0.9966 - val_precision_5: 0.9818 - val_recall_5: 0.9818
Epoch 19/100
epoch_end | time:  14.04.2021 10:33:02
1547/1547 - 1586s - loss: 0.0118 - accuracy: 0.9805 - auc_5: 0.9965 - precision_5: 0.9805 - recall_5: 0.9805 - val_loss: 0.0591 - va

In [28]:
for k_i in [5, 10]:
    for theta_i in [0.5, 1]:
        for w in [3, 5]:
            print('k=', k_i, 't=', theta_i, 'w=', w)

k= 5 t= 0.5 w= 3
k= 5 t= 0.5 w= 5
k= 5 t= 1 w= 3
k= 5 t= 1 w= 5
k= 10 t= 0.5 w= 3
k= 10 t= 0.5 w= 5
k= 10 t= 1 w= 3
k= 10 t= 1 w= 5


In [68]:
##### Дополнительный ГридСерч

# Гридсерч сетку настроить:
# k = {5, 10}
# initial_filters = {5, 10}, # количество фильтров лучше взять 5-10-20 (64 - из 2D)
# block_sizes = [3, 3] - для Spider-6; [4, 4] - для Spider-8
# conv_kernel_width = w, где for w in [5, 6] 
# bottleneck_size=2, лучше взять 2-3(общая формула Х = initial_filters + k * bottleneck_size (k={0,n}))
# transition_pool_size=2,
# transition_pool_stride=2, # здесь лучше поставить страйд = 1 (2 - из 2D)
# theta=0.5, # это параметр для баттлнек сверток 1х1, мб настроить парочку из списка: {0.4, 0.5, 0.6, 1}
# initial_conv_width=5,  # здесь лучше поставить 5 (7 - для 2D)
# initial_stride=2, # здесь лучше 1 взять (2 - из 2D)
# initial_pool_width=3, # этот пулинг лучше 2 взять (3 - из 2D)
# initial_pool_stride=2  # здесь лучше поставить страйд = 1, а то размерность будет быстро уменьшаться


n_features = 163



for k_i in [10, 15]:
    for theta_i in [0.5]:
        for w in [5, 7]:
            if ((k_i != 10) or (theta_i != 0.5) or (w != 5)):
                if True: #((k_i != 10) or (theta_i != 1) or (w != 3)):
                    model_dense = DenseNetCustom(input_shape = (163, 1),
                                                 num_outputs=2,
                                                 block_sizes= [3, 3],
                                                 initial_filters=5,
                                                 k=k_i, 
                                                 conv_kernel_width=5,
                                                 bottleneck_size=2,
                                                 transition_pool_stride=1,
                                                 theta=theta_i,
                                                 initial_conv_width=w,
                                                 initial_stride=1,
                                                 initial_pool_width=2,
                                                 initial_pool_stride=2)

                    model_dense.compile(loss='categorical_crossentropy',
                                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                                        metrics=['accuracy', tf.keras.metrics.AUC(),
                                                 tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

                    VERBOSE = 2 # формат вывода логов обучения
                    BATCH_SIZE = 512
                    NB_EPOCH = 100 # максимальное количество эпох, если не сработает EarlyStopping
                    class_weighting = {0:0.05 , 1:1} #{0:0.00163 , 1:1} можно подбирать как гипер.параметр.

                    history_logs = model_dense.fit_generator(generator=training_generator,
                                                             validation_data= (X_2_2, Y_test_2),
                                                             epochs=NB_EPOCH, verbose=VERBOSE,
                                                             class_weight=class_weighting,
                                                             callbacks=[_time, EarlyStopping(monitor='val_loss', patience=15)])

                    res_model_dense_3_3 = pd.DataFrame(history_logs.history, columns = history_logs.history.keys())
                    model_dense.save('model_dense_c3c3' + '_k' + str(k_i) + '_t' + str(theta_i) + '_w' + str(w) + '.h5')
                    res_model_dense_3_3.to_csv('model_dense_c3c3' + '_k' + str(k_i) + '_t' + str(theta_i) + '_w' + str(w) + '.csv')

                    predict_class_val = model_dense.predict(X_2_2)
                    APS = metrics.average_precision_score(y_test, predict_class_val[:,1])
                    AUC = metrics.roc_auc_score(y_test, predict_class_val[:,1])
                    GINI = 2*AUC - 1
                    print('=========================================')
                    print('VALIDATION:', 'APS:', APS, 'AUC_ROC:', AUC, 'GINI:', GINI)

                    predict_class_val = model_dense.predict(X_3_2)
                    APS = metrics.average_precision_score(y_val, predict_class_val[:,1])
                    AUC = metrics.roc_auc_score(y_val, predict_class_val[:,1])
                    GINI = 2*AUC - 1
                    print('TEST:', 'APS:', APS, 'AUC_ROC:', AUC, 'GINI:', GINI)
                    print('=========================================')
                    print('')
                    



train_begin | time:  05.04.2021 20:34:00
Epoch 1/100
epoch_end | time:  05.04.2021 21:32:47
2945/2945 - 3521s - loss: 0.0090 - accuracy: 0.9700 - auc_20: 0.9952 - precision_20: 0.9700 - recall_20: 0.9700 - val_loss: 0.0597 - val_accuracy: 0.9789 - val_auc_20: 0.9978 - val_precision_20: 0.9789 - val_recall_20: 0.9789
Epoch 2/100
epoch_end | time:  05.04.2021 22:29:00
2945/2945 - 3372s - loss: 0.0064 - accuracy: 0.9818 - auc_20: 0.9978 - precision_20: 0.9818 - recall_20: 0.9818 - val_loss: 0.1502 - val_accuracy: 0.9456 - val_auc_20: 0.9852 - val_precision_20: 0.9456 - val_recall_20: 0.9456
Epoch 3/100
epoch_end | time:  05.04.2021 23:23:57
2945/2945 - 3296s - loss: 0.0059 - accuracy: 0.9838 - auc_20: 0.9981 - precision_20: 0.9838 - recall_20: 0.9838 - val_loss: 0.1819 - val_accuracy: 0.9327 - val_auc_20: 0.9790 - val_precision_20: 0.9327 - val_recall_20: 0.9327
Epoch 4/100
epoch_end | time:  06.04.2021 00:18:30
2945/2945 - 3272s - loss: 0.0056 - accuracy: 0.9849 - auc_20: 0.9983 - precis

Epoch 31/100
epoch_end | time:  07.04.2021 01:33:25
2945/2945 - 3318s - loss: 0.0035 - accuracy: 0.9890 - auc_20: 0.9992 - precision_20: 0.9890 - recall_20: 0.9890 - val_loss: 0.0203 - val_accuracy: 0.9949 - val_auc_20: 0.9993 - val_precision_20: 0.9949 - val_recall_20: 0.9949
Epoch 32/100
epoch_end | time:  07.04.2021 02:28:48
2945/2945 - 3322s - loss: 0.0034 - accuracy: 0.9892 - auc_20: 0.9992 - precision_20: 0.9892 - recall_20: 0.9892 - val_loss: 0.0333 - val_accuracy: 0.9907 - val_auc_20: 0.9991 - val_precision_20: 0.9907 - val_recall_20: 0.9907
VALIDATION: APS: 0.05275438900597422 AUC_ROC: 0.92122279241821 GINI: 0.8424455848364201
TEST: APS: 0.04582367976578686 AUC_ROC: 0.9230978249094874 GINI: 0.8461956498189749

train_begin | time:  07.04.2021 02:30:21
Epoch 1/100
epoch_end | time:  07.04.2021 03:50:35
2945/2945 - 4809s - loss: 0.0083 - accuracy: 0.9733 - auc_21: 0.9959 - precision_21: 0.9733 - recall_21: 0.9733 - val_loss: 0.0480 - val_accuracy: 0.9849 - val_auc_21: 0.9982 - va

Epoch 28/100
epoch_end | time:  08.04.2021 16:36:14
2945/2945 - 3980s - loss: 0.0027 - accuracy: 0.9906 - auc_21: 0.9994 - precision_21: 0.9906 - recall_21: 0.9906 - val_loss: 0.0238 - val_accuracy: 0.9933 - val_auc_21: 0.9992 - val_precision_21: 0.9933 - val_recall_21: 0.9933
Epoch 29/100
epoch_end | time:  08.04.2021 17:44:59
2945/2945 - 4124s - loss: 0.0027 - accuracy: 0.9907 - auc_21: 0.9994 - precision_21: 0.9907 - recall_21: 0.9907 - val_loss: 0.0258 - val_accuracy: 0.9926 - val_auc_21: 0.9989 - val_precision_21: 0.9926 - val_recall_21: 0.9926
Epoch 30/100
epoch_end | time:  08.04.2021 19:06:37
2945/2945 - 4896s - loss: 0.0026 - accuracy: 0.9908 - auc_21: 0.9994 - precision_21: 0.9908 - recall_21: 0.9908 - val_loss: 0.0198 - val_accuracy: 0.9949 - val_auc_21: 0.9990 - val_precision_21: 0.9949 - val_recall_21: 0.9949
Epoch 31/100
epoch_end | time:  08.04.2021 20:43:13
2945/2945 - 5794s - loss: 0.0026 - accuracy: 0.9910 - auc_21: 0.9994 - precision_21: 0.9910 - recall_21: 0.9910 - 

Epoch 10/100
epoch_end | time:  10.04.2021 04:58:24
2945/2945 - 4112s - loss: 0.0042 - accuracy: 0.9882 - auc_22: 0.9990 - precision_22: 0.9882 - recall_22: 0.9882 - val_loss: 0.0295 - val_accuracy: 0.9918 - val_auc_22: 0.9991 - val_precision_22: 0.9918 - val_recall_22: 0.9918
Epoch 11/100
epoch_end | time:  10.04.2021 06:07:06
2945/2945 - 4120s - loss: 0.0040 - accuracy: 0.9885 - auc_22: 0.9990 - precision_22: 0.9885 - recall_22: 0.9885 - val_loss: 0.0204 - val_accuracy: 0.9949 - val_auc_22: 0.9992 - val_precision_22: 0.9949 - val_recall_22: 0.9949
Epoch 12/100
epoch_end | time:  10.04.2021 07:15:47
2945/2945 - 4120s - loss: 0.0039 - accuracy: 0.9886 - auc_22: 0.9991 - precision_22: 0.9886 - recall_22: 0.9886 - val_loss: 0.0221 - val_accuracy: 0.9943 - val_auc_22: 0.9993 - val_precision_22: 0.9943 - val_recall_22: 0.9943
Epoch 13/100
epoch_end | time:  10.04.2021 08:22:43
2945/2945 - 4015s - loss: 0.0039 - accuracy: 0.9888 - auc_22: 0.9991 - precision_22: 0.9888 - recall_22: 0.9888 - 